In [36]:
import pandas as pd
import os
from utils.plotter import PlotResults
from utils.utils import align_predictions
from run import *

In [37]:
argv = """
--result_path scratch
--data_path Top_100.csv 
--model Transformer --test
""".split()
parser = get_parser()
args = parser.parse_args(argv)
initial_setup(args)

In [39]:
setting = stringify_setting(args)
exp = Exp_Forecast(args, setting)

plotter = PlotResults(
    exp.output_folder, DataConfig.targets
)

Use GPU: cuda:0

Train samples 63700, validation samples 2800, test samples 2800
637 days of training, 14 days of validation data, 14 days of test data.

Fitting scalers on train data


In [47]:
flag = 'test'

if flag == 'train': df = exp.train_data
elif flag == 'val': df = exp.val_data
elif flag == 'test': df = exp.test_data
else: raise NotImplementedError

preds, trues, predictions_index = exp.pred(load_model=True, flag=flag, return_index=True)
predictions_index[exp.age_data.time_index] += args.pred_len + df[exp.age_data.time_index].min()

pred_list = [
    preds[:, :, target] for target in range(preds.shape[-1])
]

merged = align_predictions(
    df, predictions_index, 
    pred_list, exp.age_data
)
merged.head(3)

Time encoded columns : ['month', 'day', 'weekday']
Getting valid sampling locations.
loading best model from scratch\Transformer_Top_100_scale_True_mode_2\checkpoint.pth
Preds and Trues shape: (100, 14, 1) (100, 14, 1)


,Date,FIPS,TimeFromStart,Cases,Predicted_Cases
0,2021-12-12,2261,651,0.0,94.498413
1,2021-12-13,2261,652,0.0,127.520584
2,2021-12-14,2261,653,0.0,110.635696


In [42]:
merged.describe()

,FIPS,TimeFromStart,Cases,Predicted_Cases
count,62300.00000,62300.000000,62300.000000,62300.000000
mean,25826.55000,325.000000,286.210578,283.813995
std,15937.12053,179.845821,614.938618,450.233612
min,2261.00000,14.000000,0.000000,0.000000
25%,12026.00000,169.000000,36.000000,101.911423
50%,25023.00000,325.000000,122.000000,156.658493
75%,39038.50000,481.000000,318.000000,282.544960
max,55079.00000,636.000000,22286.000000,9580.916992


In [48]:
_ = plotter.summed_plot(merged, type=flag)

Target Cases: MAE 522.18, RMSE 992.94, RMSLE 3.3357, NNSE 0.5701.

